In [1]:
cd /private/home/dpf/projects/codex

/private/home/dpf/projects/codex


In [2]:
from models import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from he import make_parser, HUMAN_EVAL_STOP_WORDS
import shlex
import pprint

In [5]:
parser = make_parser()

In [6]:
args = parser.parse_args(shlex.split("--model_name /checkpoint/dpf/models/cm-6B-armen/checkpoint_last_consolidated.pt --num_candidates_generated 1 --num_candidates_evaluated 1 --batch_size 3 --temperature 0.2 --top_p 0.95 --output_filename expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/samples.jsonl --response_filename expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/responses.pkl --prompt_prefix '<| file ext=.py |>' --tokenizer_name gpt2_pretokenization_newlines_only"))

In [7]:
pprint.pprint(vars(args))

{'batch_size': 3,
 'beam': 1,
 'cached_responses': False,
 'candidate_scoring': 'mean',
 'git_status': False,
 'max_tokens': 450,
 'model_name': '/checkpoint/dpf/models/cm-6B-armen/checkpoint_last_consolidated.pt',
 'multiple_cached_responses_filenames': None,
 'num_candidates_evaluated': 1,
 'num_candidates_generated': 1,
 'num_problems': None,
 'output_filename': 'expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/samples.jsonl',
 'prompt_prefix': '<| file ext=.py |>',
 'remove_test_cases': False,
 'response_filename': 'expts/he/lm-1.3B-ourtok_last_ncg-1_temp-0.2/prompt_pyfile_normalized/responses.pkl',
 'temperature': 0.2,
 'tokenizer_name': 'gpt2_pretokenization_newlines_only',
 'top_p': 0.95,
 'verbose': False}


In [12]:
model = make_model(args)

model_root_dir: /checkpoint/dpf/models/cm-6B-armen
model_basename: checkpoint_last_consolidated.pt
dictionary size: 50518
Detected CUDA files, patching ldflags
Emitting ninja build file /private/home/dpf/projects/Megatron-LM/megatron/fused_kernels/build/build.ninja...
Building extension module fused_mix_prec_layer_norm_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module fused_mix_prec_layer_norm_cuda...


In [14]:
from priming_generator import TopPSampling, GreedyDecoding

In [17]:
decoder = TopPSampling(model.lm_model, min_len=10, max_len=128, sampling_topp=0.95, temperature=0.6, show_tqdm=True).cuda()

In [15]:
greedy = GreedyDecoding(model.lm_model, min_len=10, max_len=128, temperature=1.0, show_tqdm=True).cuda()

In [16]:
import copy

In [18]:
pdb on

Automatic pdb calling has been turned ON


In [36]:
encoded_prompt = model._encode('<| file filename=train_model.py |>\n', strip_eos=True).to(model.lm_model.device)

In [37]:
greedy.max_len = 16
rets = greedy.decode_multiple_candidates(encoded_prompt, incremental_states=None, return_incremental_states=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:01<00:00, 14.81it/s]


In [38]:
print(model._decode(rets[0].squeeze()[1:].tolist()))

<| file filename=train_model.py |>
"""
This script trains a model 


In [39]:
greedy.max_len = 32
rets_2 = greedy.decode_multiple_candidates(
    rets[0].squeeze(),
    incremental_states=rets[-1],
    return_incremental_states=True
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 14.85it/s]


In [40]:
print(model._decode(rets_2[0].squeeze()[1:].tolist()))

<| file filename=train_model.py |>
"""
This script trains a model using all available data.
"""

import os
import sys
import argparse
import importlib

